In [1]:
import PIL
from PIL import Image, ImageDraw, ImageFont, ImageChops
import textwrap

import numpy as np
import random
import os

In [2]:
def text_to_image(image, text,rectangle_area,font_size=30):

    # Create an ImageDraw object
    draw = ImageDraw.Draw(image)

    # Define the default font
    font = ImageFont.truetype("arial.ttf", font_size)

    # Define the rectangle area to add text
    x1, y1, x2, y2 = rectangle_area

    # Calculate the available width and height inside the rectangle area
    available_width = x2 - x1
    available_height = y2 - y1

    # Wrap the text to fit inside the available width
    wrapped_text = textwrap.wrap(text, width=int(available_width / font_size))

    # Calculate the total height of the wrapped text
    total_text_height = len(wrapped_text) * font_size

    # Calculate the position to center the wrapped text vertically within the rectangle area
    text_y = y1 + (available_height - total_text_height) // 2

    # Add the wrapped text to the image
    for line in wrapped_text:
        # Calculate the bounding box of the text line
        text_bbox = draw.textbbox((0, 0), line, font=font)

        # Calculate the position to center the text line horizontally within the rectangle area
        text_x = x1 + (available_width - (text_bbox[2] - text_bbox[0])) // 2

        # Draw the text line
        draw.text((text_x, text_y), line, fill="black", font=font)
        text_y += font_size

    # Save the image with the added text
    return image




In [3]:
def add_corners( im, rad=100):
    circle = Image.new('L', (rad * 2, rad * 2), 0)
    draw = ImageDraw.Draw(circle)
    draw.ellipse((0, 0, rad * 2, rad * 2), fill=255)
    alpha = Image.new('L', im.size, "white")
    w, h = im.size
    alpha.paste(circle.crop((0, 0, rad, rad)), (0, 0))
    alpha.paste(circle.crop((0, rad, rad, rad * 2)), (0, h - rad))
    alpha.paste(circle.crop((rad, 0, rad * 2, rad)), (w - rad, 0))
    alpha.paste(circle.crop((rad, rad, rad * 2, rad * 2)), (w - rad, h - rad))

    alpha = ImageChops.darker(alpha, im.split()[-1])

    im.putalpha(alpha)
    return im

def tile_to_image(image,tile_path,rectangular_area):
    tile = Image.open(img_folder+tile_path).convert("RGBA")
    # tile = add_corners(tile, 500)
    rectangular_area = np.array(rectangular_area).astype(int)


    
    # Calculate the size of the rectangle area
    insert_width = rectangular_area[2] - rectangular_area[0]
    insert_height = rectangular_area[3] - rectangular_area[1]

    # Resize the picture to fit within the specified rectangle
    resized_picture = tile.resize((int(insert_width), int(insert_height)), Image.ANTIALIAS)

    resized_picture = add_corners(resized_picture,int(0.02*image.size[0]))

    # Paste the resized picture onto the main image at the specified position
    image.paste(resized_picture, tuple(rectangular_area[:2]),resized_picture)
    return image

In [4]:
class Rectangle:
    def __init__(self, xy, text, colour, tile_image,image, type) -> None:
        self.xy = xy
        self.type = type
        
        self.text = text
        self.colour = colour
        self.tile_image = tile_image
        self.image = image
        self.edge_colour = "white"
        
    def draw(self):
        w,h = self.image.size
        draw = ImageDraw.Draw(self.image)
        
        if self.tile_image != "":
            tile_to_image(self.image,self.tile_image,self.xy)

        draw.rounded_rectangle(self.xy, fill=self.colour, outline=self.edge_colour,
                    width=3, radius=0.02*w)
        
        if self.text != "":
            text_to_image(self.image,self.text,self.xy)
        

In [5]:
class Grid:
    def __init__(self,image,metadata,NROWS,NCOLS) -> None:
        self.image = image
        self.metatada = metadata
        self.NROWS = NROWS
        self.NCOLS = NCOLS

        self.grid = [[''] * NCOLS for _ in range(NROWS)]
        self.grid_labels = [[''] * NCOLS for _ in range(NROWS)]

        self.positions = self.get_rect_positions()

    def set_info(self,song_list,img_list):
        self.song_list = song_list
        self.img_list = img_list
        

    def get_rect_positions(self):
        """Creates a grid of rectangle positions (x1,y1,x2,y2) for the given image and metadata of inner rectangle.

        Parameters
        ----------
        `metadata` : xy of inner rectangle
        `NCOLS` : Number of columns
        `NROWS` : Number of rows

        Returns
        -------
        `rect_positions` : List of all the xy positions for the rectangles in a grid
        """
        metadata,NROWS,NCOLS = self.metatada,self.NROWS,self.NCOLS
        
        N_TILES = NCOLS*NROWS
        start_x,start_y,end_x,end_y = metadata
        edge_percent = 0.005     # Percent of blank spaces on the edges
        extra_spacing = 10
        
        spacing_x = (end_x-start_x)*edge_percent
        spacing_y = (end_y-start_y)*edge_percent

        tile_w = ((end_x-start_x)-(NCOLS+extra_spacing+1)*spacing_x)/NCOLS
        tile_h = ((end_y-start_y)-(NROWS+extra_spacing+1)*spacing_y)/NROWS

        rect_positions = [[()] * NCOLS for _ in range(NROWS)]
        for i in range(NROWS):
            for j in range(NCOLS):
                x1 = start_x + (0.5*extra_spacing+1)*spacing_x + (tile_w+spacing_x)*j
                x2 = (start_x + (0.5*extra_spacing+1)*spacing_x + (tile_w+spacing_x)*(j+1)) - spacing_x
                y1 = start_y + (0.5*extra_spacing+1)*spacing_y + (tile_h+spacing_y)*i
                y2 = (start_y + (0.5*extra_spacing+1)*spacing_y + (tile_h+spacing_y)*(i+1)) - spacing_y

                rect_position = (x1,y1,x2,y2)
                rect_positions[i][j] = rect_position
                
        
        return rect_positions
        

    def fill(self, n_songs, n_img):
        ncols,nrows = self.NCOLS,self.NROWS
        total_cells = ncols*nrows
        n_blanks = total_cells - n_songs - n_img

        total_classes = n_songs + n_img + n_blanks

        if total_classes > total_cells:
            raise ValueError("The total number of classes exceeds the available grid cells.")

        # Calculate the number of instances of class A in each row
        songs_per_row = n_songs // nrows

        # Create a list to keep track of the row indices and shuffle it
        row_indices = list(range(nrows))
        random.shuffle(row_indices)

        # Create an empty grid to store the classes
        grid = [[''] * ncols for _ in range(nrows)]

        # Distribute class A to the grid, ensuring each row has the same number of instances
        count = 0
        for row in row_indices:
            for i in range(songs_per_row):
                col = random.randint(0, ncols - 1)
                while grid[row][col]:
                    col = random.randint(0, ncols - 1)
                 
                grid[row][col] = Rectangle(self.positions[row][col],text=self.song_list[count],colour=LBLUE,tile_image="",image=self.image,type="Song")
                self.grid_labels[row][col] = "S"
                count += 1

        # Randomly distribute class B to the grid
        for i in range(n_img):
            row, col = random.randint(0, nrows - 1), random.randint(0, ncols - 1)
            while grid[row][col]:
                row, col = random.randint(0, nrows - 1), random.randint(0, ncols - 1)
            grid[row][col] = Rectangle(self.positions[row][col],text="",colour=None,tile_image=self.img_list[i],image=self.image,type="Image")
            self.grid_labels[row][col] = "I"

        # Randomly distribute class C to the grid
        for i in range(n_blanks):
            row, col = random.randint(0, nrows - 1), random.randint(0, ncols - 1)
            while grid[row][col]:
                row, col = random.randint(0, nrows - 1), random.randint(0, ncols - 1)
            grid[row][col] = Rectangle(self.positions[row][col],text="",colour=BLAU,tile_image="",image=self.image,type="Blank")
            self.grid_labels[row][col] = "B"
        
        self.grid = grid
        return grid
    
    def log(self,mode="type"):

        if mode.lower() == "type":
            for row in self.grid_labels: print(row)
        elif mode.lower() == "object":
            for row in self.grid: print(row)
        
    
    def draw(self):
        w,h = self.image.size

        draw = ImageDraw.Draw(self.image)
        for row in self.grid:
            for Rect in row:
                Rect.draw()
        return self.image

In [6]:
def check_repeated(list,sublists,song_list,N_SONGS_BILL):
    for sl in sublists:
        if set(list) == set(sl):
            list = random.sample(song_list, N_SONGS_BILL)
            break
    return list


def generate_sublists(song_list, N, M):

    # Create a list to store the sublists
    sublists = []

    # Shuffle the big list to randomize the order of elements
    random.shuffle(song_list)

    # Divide the big list into N sublists of M elements each
    for i in range(N):
        sublist = random.sample(song_list, M)
        sublist = check_repeated(sublist,sublists,song_list,M)

        sublists.append(sublist)

    return sublists

In [7]:
def background(image:Image, fills:list, edges:list):
    """Generates background of billet onto a given PIL image with the selected list of
    fill and edge colours of rounded rectangles (from outside to inside).

    Parameters
    ----------
    `image` : PIL image
    `fills` : List of fill colours for the rounded rectangles, from outer to inner
    `edges` : List of edge colours for the rounded rectangles, from outer to inner

    Returns
    -------
    `image` : Drawn image with background
    `metadata` : Pixel data of the last (inner) rectangle position
    """

    edge_percent = 0.04     # Percent of blank spaces on the edges
    
    w,h = image.size
    draw = ImageDraw.Draw(image)

    # Draw rounded rectangles
    for i,(fill,edge) in enumerate(zip(fills,edges)):
        ep_i = edge_percent * (i+1) * 0.4
        draw.rounded_rectangle((ep_i*w, ep_i*h, (1-ep_i)*w, (1-ep_i)*h), fill=fill, outline=edge,
                            width=3, radius=edge_percent*w)
        
    metadata = np.array((ep_i*w, ep_i*h, (1-ep_i)*w, (1-ep_i)*h)).astype(int)
    return image, metadata


In [16]:
def read_song_fie(song_file):    
    song_list = []
    with open(song_file,"r") as inFile: 
        for line in inFile:
            data = line.split("-")
            artist,song = data[0].strip(),data[1].strip()
            text = f"{artist} – {song}"
            song_list.append(text)
    return song_list

In [8]:
def ensure_elements_present(bills, required_elements):
   
    for i,input_list in enumerate(bills):
        # Count the occurrences of required elements in the input list
        element_counts = {element: input_list.count(element) for element in required_elements}
        
        # Add missing elements with counts calculated to maintain the original list length
        for element, count in element_counts.items():
            while count < 1:
                for i, item in enumerate(input_list):
                    if item not in required_elements:
                        input_list[i] = element
                        count += 1
                        break
        bills[i] = input_list
    return bills


In [17]:
N_SONGS = 50
BLAU = (0,77,152)
GRANA = (165,0,68)
LBLUE = (175,215,255)

# User input
N_PLAYERS = 20
N_SONGS_BILL = 12
N_IMAGES_BILL = 10
NROWS, NCOLS = 3, 9

fills = [BLAU,GRANA,BLAU]
edges = [BLAU,GRANA,BLAU]

times = 20
WIDTH, HEIGHT = 160*times,70*times

img_folder = "./img/"
img_list = os.listdir(img_folder)
# Song list parsing
SONGS_FILE = "song_list.txt"
song_list = read_song_fie(SONGS_FILE)

songs_per_bill = generate_sublists(song_list, N_PLAYERS, N_SONGS_BILL)
images_per_bill = generate_sublists(img_list,N_PLAYERS,N_IMAGES_BILL)

required_images = ["fran.jpg","raquel.jpg","emma.jpg"]
images_per_bill = ensure_elements_present(images_per_bill, required_images)

try: os.mkdir("bills/")
except FileExistsError: pass

for i in range(N_PLAYERS):
    bg_colour = "white"
    image = Image.new("RGBA", (WIDTH, HEIGHT), bg_colour)

    image, metadata = background(image,fills,edges)

    song_list_i = songs_per_bill[i]
    img_list = images_per_bill[i]

    grid = Grid(image,metadata,NROWS,NCOLS)
    grid.set_info(song_list_i,img_list)
    grid.fill(N_SONGS_BILL,N_IMAGES_BILL)
    # grid.log()
    image = grid.draw()
    image.save(f"bills/bill{i}.png",dpi=(1200,1200))



C:\Users\diego\AppData\Local\Temp\ipykernel_22840\463286950.py:29: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resized_picture = tile.resize((int(insert_width), int(insert_height)), Image.ANTIALIAS)


In [32]:
def read_input(input_file):
    """Reads and parses song file"""  
    parameters = {}
    with open(input_file,"r") as inFile: 
        for line in inFile:
            if line.strip() == "": continue
            elif line.strip().startswith("#"): continue
            key_value, comment = line.strip().split('#', 1) if '#' in line else (line.strip(), '')
            key, value = key_value.split('=', 1)
            parameters[key.strip()] = value.strip()

    return parameters

read_input("input.txt")

{'NROWS': '3',
 'NCOLS': '9',
 'N_PLAYERS': '10',
 'N_SONGS_CARD': '12',
 'N_IMAGES_CARD': '10',
 'WIDTH': '16',
 'HEIGHT': '7',
 'TIMES': '200',
 'IMG_FOLDER': 'img/',
 'SONGS_FILE': 'songs.txt',
 'OUTPUT_FOLDER': 'bills/',
 'BG_COLOUR': 'white',
 'FILLS': 'blue, red, blue',
 'EDGES': 'same',
 'TILE_COLOUR': 'green'}